In [1]:
import openai

In [2]:
from openai import OpenAI

In [3]:
client = OpenAI()

In [6]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[{"role": "user", "content": "is it too late to join the course?"}]
)

In [7]:
response.choices[0].message.content

"It depends on the specific course and its enrollment policies. It's best to check with the course provider or institution to see if they are still accepting new students."